In [1]:
import sys
sys.path.insert(0, "../")
from tmds import *
import os

2.7.12 (default, Nov 12 2018, 14:36:49) 
[GCC 5.4.0 20160609]


In [10]:
s = ["SRFA"]
cities = list(map(lambda x: str(x), range(1, 10)))

In [11]:
elems = "C H O N S".split()

In [13]:
data = []
for i in s:
    data.append([])
    for j in cities:
        data[-1].append(pd.read_csv("All-SRFA-Formulae/{}{}.txt".format(i, j), sep='\t'))
        data[-1][-1] = data[-1][-1][data[-1][-1]["formula"].str.contains("13C", regex=False) != True]
        if "Unnamed: 0" in list(data[-1][-1]):
            data[-1][-1] = data[-1][-1].drop("Unnamed: 0", axis=1)

In [14]:
data[0][0].head()

,peakNo,mw,relativeAbundance,errorPPM,DBE,class,formula,z,C,H,O,N,S
0,531,343.054026,0.008016,0.233625,7,N1O10,H13_C13_N_O10,0,13,13,10,1,0
1,625,353.038416,0.011549,0.340530,9,N1O10,H11_C14_N_O10,0,14,11,10,1,0
2,644,355.054026,0.017521,0.225729,8,N1O10,H13_C14_N_O10,0,14,13,10,1,0
3,666,357.069676,0.016318,0.224248,7,N1O10,H15_C14_N_O10,0,14,15,10,1,0
4,731,363.022536,0.010643,-0.302202,11,N1O10,H9_C15_N_O10,0,15,9,10,1,0


In [16]:
abundance = []
for i in range(1):
    abundance.append([])
    for j in range(9):
        abundance[-1].append(get_dict_of_set(data[i][j], value="relativeAbundance", error="errorPPM"))

In [17]:
error = []
for i in range(1):
    error.append([])
    for j in range(9):
        error[-1].append(get_dict_of_set(data[i][j], value="errorPPM", error="errorPPM"))

In [18]:
mw = []
for i in range(1):
    mw.append([])
    for j in range(9):
        mw[-1].append(get_dict_of_set(data[i][j], value="mw", error="errorPPM"))

In [19]:
tuples = []
for i in range(1):
    tuples.append([])
    for j in range(9):
        tuples[-1].append(get_set_of_tuples_from_np(data[i][j][elems].values))

In [21]:
def get_inter_residues(n, tuples=tuples, shape=(6, 5)):
    tf = []
    for i in range(1):
        tf.append(set([]))
        for j in combinations(range(9), n):
            t = reduce(lambda x, y: x & y, map(lambda x: tuples[i][x], j))
            tf[-1] |= t

    inter = []
    for i in range(1):
        inter.append([])
        for j in range(9):
            inter[-1].append(tf[i] & tuples[i][j])

    residue = []
    for i in range(1):
        residue.append([])
        for j in range(9):
            residue[-1].append(tuples[i][j] - tf[i])
        
    return inter, residue

In [22]:
inter = []
residue = []
for n in range(1, 10):
    i, r = get_inter_residues(n)
    inter.append(i)
    residue.append(r)

In [23]:
inter[4][0][4]

{(30, 32, 19, 0, 0),
 (33, 38, 18, 0, 0),
 (27, 34, 10, 0, 0),
 (14, 17, 9, 1, 0),
 (29, 34, 16, 0, 0),
 (13, 18, 4, 0, 0),
 (12, 14, 6, 0, 0),
 (24, 26, 13, 0, 0),
 (16, 26, 9, 0, 0),
 (32, 26, 20, 0, 0),
 (16, 12, 9, 0, 0),
 (17, 14, 4, 0, 0),
 (31, 38, 18, 0, 0),
 (26, 40, 10, 0, 0),
 (20, 14, 13, 0, 0),
 (27, 38, 11, 0, 0),
 (18, 23, 8, 1, 0),
 (30, 32, 14, 0, 0),
 (19, 22, 14, 0, 0),
 (16, 22, 8, 0, 0),
 (10, 6, 7, 0, 0),
 (17, 13, 9, 1, 0),
 (17, 28, 5, 0, 0),
 (18, 22, 14, 0, 0),
 (23, 32, 14, 0, 0),
 (25, 24, 13, 0, 0),
 (19, 23, 12, 1, 0),
 (15, 17, 10, 1, 0),
 (33, 46, 14, 0, 0),
 (15, 26, 6, 0, 0),
 (21, 12, 13, 0, 0),
 (21, 26, 5, 0, 0),
 (11, 12, 7, 0, 0),
 (24, 12, 15, 0, 0),
 (36, 34, 19, 0, 0),
 (25, 28, 10, 0, 0),
 (29, 30, 16, 0, 0),
 (38, 34, 21, 0, 0),
 (26, 38, 8, 0, 0),
 (27, 40, 13, 0, 0),
 (20, 10, 11, 0, 0),
 (18, 16, 16, 0, 0),
 (21, 8, 10, 0, 0),
 (18, 19, 6, 1, 0),
 (17, 18, 10, 2, 0),
 (35, 40, 13, 0, 0),
 (19, 26, 8, 0, 0),
 (30, 26, 20, 0, 0),
 (20, 24, 1

In [25]:
max_inter = []
for i in range(1):
    max_inter.append([])
    for j in range(9):
        max_inter[-1].append({})
        
for k in range(9):
    for i in range(1):
        for j in range(9):
            for t in inter[k][i][j]:
                max_inter[i][j][t] = k + 1

In [26]:
st = "exp\ttown\tabundance\tmw\terror\tC\tH\tO\tN\tS\ty\n"
for i in range(1):
    for j in range(9):
        I = list(map(lambda x: abundance[i][j][x], tuples[i][j]))        
        E = list(map(lambda x: error[i][j][x], tuples[i][j]))
        M = list(map(lambda x: mw[i][j][x], tuples[i][j]))
        F = list(map(lambda x: x, tuples[i][j]))
        Y = list(map(lambda x: max_inter[i][j][x], tuples[i][j]))
        for ii, e, m, f, y in zip(I, E, M, F, Y):
            t = '\t'.join(map(lambda x: str(x), f))
            #st += f"{s[i]}\t{cities[j]}\t{ii}\t{m}\t{e}\t{t}\t{1}\n"
            st += "{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(s[i], cities[j], ii, m, e, t, y)
            

In [27]:
with open("SRFA.csv", "w") as f:
    f.write(st)